In [1]:
import random

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
import pickle
import sklearn.metrics as metrics

### functions

In [2]:
def edge_features(node_emb_1, node_emb_2, operator):
    node_emb_1 = np.asfarray(node_emb_1,float)
    node_emb_2 = np.asfarray(node_emb_2, float)
    # combine two nodes' embeddings with specificed operator
    if operator == 'Average':
        edge = [((x + y) / 2.0) for x,y in zip(node_emb_1, node_emb_2)]
    elif operator == 'Hadamard':
        edge = [(x * y) for x,y in zip(node_emb_1, node_emb_2)]
    elif operator == 'Weighted-L1':
        edge = [abs(x - y) for x,y in zip(node_emb_1, node_emb_2)]
    elif operator == 'Weighted-L2':
        edge = [abs(x - y)**2 for x,y in zip(node_emb_1, node_emb_2)]
    elif operator == 'Concat':
        edge = np.concatenate((node_emb_1, node_emb_2), axis=None) 
    else:
        print("Generate edge features: Operator not supported")
        print("Use default operator: Weighted-L1")
        edge = [abs(x - y) for x,y in zip(node_emb_1, node_emb_2)]
        
    return edge
def generate_edge_features(edge_list, node_embeddings, operator):
    edge_features_mtx = []
    
    # generate features for each edge in the list
    for node_index_1, node_index_2 in edge_list:
        node_emb_1 = node_embeddings[node_index_1-1]
        node_emb_2 = node_embeddings[node_index_2-1]
        
        edge_features_mtx.append(edge_features(node_emb_1, node_emb_2, operator))
        
    return edge_features_mtx

def generate_train_set(graph_train, num_edge_sample, node_embeddings, edge_operator,):
    edge_list = list(graph_train.edges)
    num_nodes = graph_train.number_of_nodes()
    
    train_edges = []
    train_edges_labels = [1] * num_edge_sample + [0] * num_edge_sample
    
    random.seed(0)
    
    # sample edges with label 1 (true edges)
    for edge_num in range(num_edge_sample):
        rand_index = random.randint(0, len(edge_list) - 1)
        
        #train_edges.append(tuple(edge_list[rand_index]))
        train_edges.append(edge_list[rand_index])
    non_edge_num = 0
    
    # sample edges with label 0 (non-exist edges)
    while(non_edge_num < num_edge_sample):
        rand_nodes = tuple(np.random.randint(low=0,high=num_nodes, size=2))
        
        if rand_nodes not in edge_list:
            train_edges.append(rand_nodes)
            non_edge_num += 1

    train_edges_features_mtx = generate_edge_features(train_edges, node_embeddings, edge_operator)
            
    return train_edges, train_edges_features_mtx, train_edges_labels

def generate_test_set(graph_test, node_embeddings, edge_operator):
    edge_list = graph_test.edges
    nodes_with_edge = set()
    
    for edge in edge_list:
        nodes_with_edge.add(edge[0])
        nodes_with_edge.add(edge[1])
    
    num_nodes = graph_test.number_of_nodes()
    
    test_edges = []
    test_edges_labels = []
    
    num_edge_sample = len(edge_list)
    non_edge_num = 0 
    # sample edges with label 0 (non-exist edges)
    
    while(non_edge_num < num_edge_sample):
        rand_nodes = tuple(np.random.randint(low=0,high=num_nodes, size=2))
        
        if rand_nodes not in edge_list:
            test_edges.append(rand_nodes)
            test_edges_labels.append(0)
            non_edge_num += 1
        
    for edge in edge_list:
        test_edges.append(edge)
        test_edges_labels.append(1)
    
    test_edges_features_mtx = generate_edge_features(test_edges, node_embeddings, edge_operator)
    
    return test_edges, test_edges_features_mtx, test_edges_labels

def build_clf(feature_mtx, response_vec):
   
    logistic_regression_model = LogisticRegression(random_state = 0,max_iter=5000,solver='liblinear',verbose=1,tol=1e-6)
    binary_clf = logistic_regression_model.fit(feature_mtx, response_vec)
    
    return binary_clf

def pred_links(feature_mtx, LR_clf):
    predict_edges_labels = LR_clf.predict(feature_mtx)
    
    return predict_edges_labels

def precision_recall(predict_labels, true_labels):
    true_positive  = false_positive = 0
    true_negative =  false_negative = 0
    
    for p_label, true_label in zip(predict_labels, true_labels):
        
        #print(p_label,true_label)
        if p_label == true_label and true_label == 1:
            true_positive += 1
        elif p_label == true_label and true_label == 0:
            true_negative += 1
        elif p_label != true_label and true_label == 1:
            false_negative += 1
        elif p_label != true_label and true_label == 0:
            false_positive += 1

    print("TP: ", true_positive)
    print("TN: ", true_negative)
    print("FP: ", false_positive)
    print("FN: ", false_negative)
    
    precision = true_positive / (true_positive + false_positive)
    recall = true_positive / (true_positive + false_negative)
    if (precision + recall) != 0.0:
        f1 = 2 * (precision * recall) / (precision + recall)
        print("F1: {}".format(f1))
    else:
        print("F1: Divide-by-zero")
    
    cm = metrics.confusion_matrix(true_labels, predict_labels)
    print(cm)
    print(metrics.classification_report(true_labels, predict_labels))
    map = metrics.average_precision_score(true_labels, predict_labels)
    print('Mean Average Precision: {}'.format(map))
    fpr, tpr, thresholds = metrics.roc_curve(true_labels, predict_labels)
    roc_auc = metrics.auc(fpr, tpr)
    print('Area Under ROC Curve: {}'.format(roc_auc))
    
    return precision, recall

### Email-EU - Monthly
### load data

In [3]:
# load the training and testing graph
with open('../data/email_EU/email_1_month_dir.pkl', 'rb') as file:
    graphs = pickle.load(file)
graph_train = graphs[-2]
graph_test = graphs[-1]

# parameters
num_edge_sample = 400

In [5]:
num_snapshots = len(graphs)
num_snapshots

17

In [6]:
emb_list = []
for i in range(num_snapshots):
    file = '../data/email_EU/embeddings/em-email-month-' + str(i) + '.npy'
    node_embedding = np.load(file)
    emb_list.append(node_embedding)
print(len(emb_list))

17


### independent / no combination

In [7]:
node_embeddings_training = emb_list[-3]
node_embeddings_testing = emb_list[-2]

In [8]:
for edge_operator in ['Average','Hadamard','Weighted-L1','Weighted-L2', 'Concat']:
    # generate the training set
    train_edges, train_edges_features_mtx, train_edges_labels = generate_train_set(graph_train, num_edge_sample, node_embeddings_training, edge_operator)
    # generate the testing set
    test_edges, test_edges_features_mtx, test_edges_labels = generate_test_set(graph_test, node_embeddings_testing, edge_operator)

    LR_clf = build_clf(train_edges_features_mtx, train_edges_labels)

    print("Edge Operator: {}".format(edge_operator))
    predict_edges_labels = pred_links(test_edges_features_mtx, LR_clf)
    precision, recall = precision_recall(list(predict_edges_labels), list(test_edges_labels))
    print('Precision: ', precision)
    print('Recall: ', recall)

[LibLinear]Edge Operator: Average
TP:  1059
TN:  5855
FP:  1171
FN:  5967
F1: 0.22882454624027662
[[5855 1171]
 [5967 1059]]
              precision    recall  f1-score   support

           0       0.50      0.83      0.62      7026
           1       0.47      0.15      0.23      7026

   micro avg       0.49      0.49      0.49     14052
   macro avg       0.49      0.49      0.43     14052
weighted avg       0.49      0.49      0.43     14052

Mean Average Precision: 0.49621495559733925
Area Under ROC Curve: 0.4920296043267863
Precision:  0.4748878923766816
Recall:  0.15072587532023912
[LibLinear]Edge Operator: Hadamard
TP:  1923
TN:  5029
FP:  1997
FN:  5103
F1: 0.3513612278457884
[[5029 1997]
 [5103 1923]]
              precision    recall  f1-score   support

           0       0.50      0.72      0.59      7026
           1       0.49      0.27      0.35      7026

   micro avg       0.49      0.49      0.49     14052
   macro avg       0.49      0.49      0.47     14052
weight

### sum

In [9]:
for i in range(len(emb_list)):
    emb_list[i] = np.asfarray(emb_list[i],float)
node_embeddings_training = np.sum(np.asarray(emb_list[0:-2]),axis=0)
node_embeddings_testing = np.sum(np.asarray(emb_list[0:-1]),axis=0)

In [10]:
for edge_operator in ['Average', 'Hadamard','Weighted-L1','Weighted-L2', 'Concat']:
    # generate the training set
    train_edges, train_edges_features_mtx, train_edges_labels = generate_train_set(graph_train, num_edge_sample, node_embeddings_training, edge_operator)
    # generate the testing set
    test_edges, test_edges_features_mtx, test_edges_labels = generate_test_set(graph_test, node_embeddings_testing, edge_operator)

    LR_clf = build_clf(train_edges_features_mtx, train_edges_labels)

    print("Edge Operator: {}".format(edge_operator))
    predict_edges_labels = pred_links(test_edges_features_mtx, LR_clf)
    precision, recall = precision_recall(list(predict_edges_labels), list(test_edges_labels))
    print('Precision: ', precision)
    print('Recall: ', recall)

[LibLinear]Edge Operator: Average
TP:  5159
TN:  3275
FP:  3751
FN:  1867
F1: 0.6474648594377511
[[3275 3751]
 [1867 5159]]
              precision    recall  f1-score   support

           0       0.64      0.47      0.54      7026
           1       0.58      0.73      0.65      7026

   micro avg       0.60      0.60      0.60     14052
   macro avg       0.61      0.60      0.59     14052
weighted avg       0.61      0.60      0.59     14052

Mean Average Precision: 0.5580166085052697
Area Under ROC Curve: 0.6001992598918304
Precision:  0.5790123456790124
Recall:  0.7342727013948193
[LibLinear]Edge Operator: Hadamard
TP:  3826
TN:  3564
FP:  3462
FN:  3200
F1: 0.5345815285734246
[[3564 3462]
 [3200 3826]]
              precision    recall  f1-score   support

           0       0.53      0.51      0.52      7026
           1       0.52      0.54      0.53      7026

   micro avg       0.53      0.53      0.53     14052
   macro avg       0.53      0.53      0.53     14052
weighted 

### expdecay

In [17]:
exps = [np.exp(-i * 0.3) for i in range(1,17)]
node_embeddings_training = np.zeros((emb_list[0]).shape) 
for c,e in zip(emb_list[0:-2],exps[:-1]):
    node_embeddings_training += e * c

In [18]:
node_embeddings_testing = np.zeros((emb_list[0]).shape) 
for c,e in zip(emb_list[0:-1],exps[:-1]):
    node_embeddings_testing += e * c 

In [19]:
for edge_operator in [ 'Average', 'Hadamard','Weighted-L1','Weighted-L2', 'Concat']:
    # generate the training set
    train_edges, train_edges_features_mtx, train_edges_labels = generate_train_set(graph_train, num_edge_sample, node_embeddings_training, edge_operator)
    # generate the testing set
    test_edges, test_edges_features_mtx, test_edges_labels = generate_test_set(graph_test, node_embeddings_testing, edge_operator)

    LR_clf = build_clf(train_edges_features_mtx, train_edges_labels)

    print("Edge Operator: {}".format(edge_operator))
    predict_edges_labels = pred_links(test_edges_features_mtx, LR_clf)
    precision, recall = precision_recall(list(predict_edges_labels), list(test_edges_labels))
    print('Precision: ', precision)
    print('Recall: ', recall)

[LibLinear]Edge Operator: Average
TP:  4111
TN:  4263
FP:  2763
FN:  2915
F1: 0.5915107913669065
[[4263 2763]
 [2915 4111]]
              precision    recall  f1-score   support

           0       0.59      0.61      0.60      7026
           1       0.60      0.59      0.59      7026

   micro avg       0.60      0.60      0.60     14052
   macro avg       0.60      0.60      0.60     14052
weighted avg       0.60      0.60      0.60     14052

Mean Average Precision: 0.5573706407084671
Area Under ROC Curve: 0.5959294050668944
Precision:  0.5980506255455339
Recall:  0.5851124395103899
[LibLinear]Edge Operator: Hadamard
TP:  4663
TN:  2567
FP:  4459
FN:  2363
F1: 0.5775328214020311
[[2567 4459]
 [2363 4663]]
              precision    recall  f1-score   support

           0       0.52      0.37      0.43      7026
           1       0.51      0.66      0.58      7026

   micro avg       0.51      0.51      0.51     14052
   macro avg       0.52      0.51      0.50     14052
weighted 

### Email-EU - Weekly
### load data

In [21]:
# load the training and testing graph
with open('../data/email_EU/email_1_week_dir.pkl', 'rb') as file:
    graphs = pickle.load(file)
graph_train = graphs[-2]
graph_test = graphs[-1]

# parameters
num_edge_sample = 400

In [22]:
num_snapshots = len(graphs)
num_snapshots

74

In [23]:
emb_list = []
for i in range(num_snapshots):
    file = '../data/email_EU/embeddings/em-email-week-' + str(i) + '.npy'
    node_embedding = np.load(file)
    emb_list.append(node_embedding)
print(len(emb_list))

74


### independent / no combination

In [24]:
node_embeddings_training = emb_list[-3]
node_embeddings_testing = emb_list[-2]

In [25]:
for edge_operator in ['Average', 'Hadamard','Weighted-L1','Weighted-L2', 'Concat']:
    # generate the training set
    train_edges, train_edges_features_mtx, train_edges_labels = generate_train_set(graph_train, num_edge_sample, node_embeddings_training, edge_operator)
    # generate the testing set
    test_edges, test_edges_features_mtx, test_edges_labels = generate_test_set(graph_test, node_embeddings_testing, edge_operator)

    LR_clf = build_clf(train_edges_features_mtx, train_edges_labels)

    print("Edge Operator: {}".format(edge_operator))
    predict_edges_labels = pred_links(test_edges_features_mtx, LR_clf)
    precision, recall = precision_recall(list(predict_edges_labels), list(test_edges_labels))
    print('Precision: ', precision)
    print('Recall: ', recall)

[LibLinear]Edge Operator: Average
TP:  758
TN:  1905
FP:  780
FN:  1927
F1: 0.35898650248638403
[[1905  780]
 [1927  758]]
              precision    recall  f1-score   support

           0       0.50      0.71      0.58      2685
           1       0.49      0.28      0.36      2685

   micro avg       0.50      0.50      0.50      5370
   macro avg       0.49      0.50      0.47      5370
weighted avg       0.49      0.50      0.47      5370

Mean Average Precision: 0.49798088402312124
Area Under ROC Curve: 0.4959031657355679
Precision:  0.4928478543563069
Recall:  0.2823091247672253
[LibLinear]Edge Operator: Hadamard
TP:  1765
TN:  1024
FP:  1661
FN:  920
F1: 0.5776468663066602
[[1024 1661]
 [ 920 1765]]
              precision    recall  f1-score   support

           0       0.53      0.38      0.44      2685
           1       0.52      0.66      0.58      2685

   micro avg       0.52      0.52      0.52      5370
   macro avg       0.52      0.52      0.51      5370
weighted a

### sum

In [26]:
for i in range(len(emb_list)):
    emb_list[i] = np.asfarray(emb_list[i],float)
node_embeddings_training = np.sum(np.asarray(emb_list[0:-2]),axis=0)
node_embeddings_testing = np.sum(np.asarray(emb_list[0:-1]),axis=0)

In [27]:
for edge_operator in ['Average', 'Hadamard','Weighted-L1','Weighted-L2', 'Concat']:
    # generate the training set
    train_edges, train_edges_features_mtx, train_edges_labels = generate_train_set(graph_train, num_edge_sample, node_embeddings_training, edge_operator)
    # generate the testing set
    test_edges, test_edges_features_mtx, test_edges_labels = generate_test_set(graph_test, node_embeddings_testing, edge_operator)

    LR_clf = build_clf(train_edges_features_mtx, train_edges_labels)

    print("Edge Operator: {}".format(edge_operator))
    predict_edges_labels = pred_links(test_edges_features_mtx, LR_clf)
    precision, recall = precision_recall(list(predict_edges_labels), list(test_edges_labels))
    print('Precision: ', precision)
    print('Recall: ', recall)

[LibLinear]Edge Operator: Average
TP:  1672
TN:  1563
FP:  1122
FN:  1013
F1: 0.610330352254061
[[1563 1122]
 [1013 1672]]
              precision    recall  f1-score   support

           0       0.61      0.58      0.59      2685
           1       0.60      0.62      0.61      2685

   micro avg       0.60      0.60      0.60      5370
   macro avg       0.60      0.60      0.60      5370
weighted avg       0.60      0.60      0.60      5370

Mean Average Precision: 0.5612912212789044
Area Under ROC Curve: 0.6024208566108008
Precision:  0.5984251968503937
Recall:  0.6227188081936685
[LibLinear]Edge Operator: Hadamard
TP:  1382
TN:  1425
FP:  1260
FN:  1303
F1: 0.5188661535573493
[[1425 1260]
 [1303 1382]]
              precision    recall  f1-score   support

           0       0.52      0.53      0.53      2685
           1       0.52      0.51      0.52      2685

   micro avg       0.52      0.52      0.52      5370
   macro avg       0.52      0.52      0.52      5370
weighted a

### expdecay

In [41]:
exps = [np.exp(-i * 0.3) for i in range(1,74)]
node_embeddings_training = np.zeros((emb_list[0]).shape) 
for c,e in zip(emb_list[0:-2],exps[:-1]):
    node_embeddings_training += e * c 
node_embeddings_testing = np.zeros((emb_list[0]).shape) 
for c,e in zip(emb_list[0:-1],exps[:-1]):
    node_embeddings_testing += e * c 

In [42]:
for edge_operator in ['Average', 'Hadamard','Weighted-L1','Weighted-L2', 'Concat']:
    # generate the training set
    train_edges, train_edges_features_mtx, train_edges_labels = generate_train_set(graph_train, num_edge_sample, node_embeddings_training, edge_operator)
    # generate the testing set
    test_edges, test_edges_features_mtx, test_edges_labels = generate_test_set(graph_test, node_embeddings_testing, edge_operator)

    LR_clf = build_clf(train_edges_features_mtx, train_edges_labels)

    print("Edge Operator: {}".format(edge_operator))
    predict_edges_labels = pred_links(test_edges_features_mtx, LR_clf)
    precision, recall = precision_recall(list(predict_edges_labels), list(test_edges_labels))
    print('Precision: ', precision)
    print('Recall: ', recall)

[LibLinear]Edge Operator: Average
TP:  1378
TN:  1756
FP:  929
FN:  1307
F1: 0.5520833333333333
[[1756  929]
 [1307 1378]]
              precision    recall  f1-score   support

           0       0.57      0.65      0.61      2685
           1       0.60      0.51      0.55      2685

   micro avg       0.58      0.58      0.58      5370
   macro avg       0.59      0.58      0.58      5370
weighted avg       0.59      0.58      0.58      5370

Mean Average Precision: 0.5499428909988949
Area Under ROC Curve: 0.5836126629422719
Precision:  0.5973125270914608
Recall:  0.5132216014897579
[LibLinear]Edge Operator: Hadamard
TP:  1947
TN:  878
FP:  1807
FN:  738
F1: 0.604752290728374
[[ 878 1807]
 [ 738 1947]]
              precision    recall  f1-score   support

           0       0.54      0.33      0.41      2685
           1       0.52      0.73      0.60      2685

   micro avg       0.53      0.53      0.53      5370
   macro avg       0.53      0.53      0.51      5370
weighted avg 

### Email-EU - Equal Monthly
### load data

In [43]:
# load the training and testing graph
with open('../data/email_EU/email_equal_monthly_dir.pkl', 'rb') as file:
    graphs = pickle.load(file)
graph_train = graphs[-2]
graph_test = graphs[-1]

# parameters
num_edge_sample = 400

In [44]:
num_snapshots = len(graphs)
num_snapshots

17

In [45]:
emb_list = []
for i in range(num_snapshots):
    file = '../data/email_EU/embeddings/em-email-equal-month-' + str(i) + '.npy'
    node_embedding = np.load(file)
    emb_list.append(node_embedding)
print(len(emb_list))

17


### independent / no combination

In [46]:
node_embeddings_training = emb_list[-3]
node_embeddings_testing = emb_list[-2]

In [47]:
for edge_operator in ['Average','Hadamard','Weighted-L1','Weighted-L2', 'Concat']:
    # generate the training set
    train_edges, train_edges_features_mtx, train_edges_labels = generate_train_set(graph_train, num_edge_sample, node_embeddings_training, edge_operator)
    # generate the testing set
    test_edges, test_edges_features_mtx, test_edges_labels = generate_test_set(graph_test, node_embeddings_testing, edge_operator)

    LR_clf = build_clf(train_edges_features_mtx, train_edges_labels)

    print("Edge Operator: {}".format(edge_operator))
    predict_edges_labels = pred_links(test_edges_features_mtx, LR_clf)
    precision, recall = precision_recall(list(predict_edges_labels), list(test_edges_labels))
    print('Precision: ', precision)
    print('Recall: ', recall)

[LibLinear]Edge Operator: Average
TP:  3685
TN:  2090
FP:  3239
FN:  1644
F1: 0.6014853505264017
[[2090 3239]
 [1644 3685]]
              precision    recall  f1-score   support

           0       0.56      0.39      0.46      5329
           1       0.53      0.69      0.60      5329

   micro avg       0.54      0.54      0.54     10658
   macro avg       0.55      0.54      0.53     10658
weighted avg       0.55      0.54      0.53     10658

Mean Average Precision: 0.5222709927119078
Area Under ROC Curve: 0.5418465002814787
Precision:  0.532206816868862
Recall:  0.6914993432163633
[LibLinear]Edge Operator: Hadamard
TP:  3678
TN:  1713
FP:  3616
FN:  1651
F1: 0.5827457815099422
[[1713 3616]
 [1651 3678]]
              precision    recall  f1-score   support

           0       0.51      0.32      0.39      5329
           1       0.50      0.69      0.58      5329

   micro avg       0.51      0.51      0.51     10658
   macro avg       0.51      0.51      0.49     10658
weighted a

### sum

In [48]:
for i in range(len(emb_list)):
    emb_list[i] = np.asfarray(emb_list[i],float)
node_embeddings_training = np.sum(np.asarray(emb_list[0:-2]),axis=0)
node_embeddings_testing = np.sum(np.asarray(emb_list[0:-1]),axis=0)

In [49]:
for edge_operator in ['Average', 'Hadamard','Weighted-L1','Weighted-L2', 'Concat']:
    # generate the training set
    train_edges, train_edges_features_mtx, train_edges_labels = generate_train_set(graph_train, num_edge_sample, node_embeddings_training, edge_operator)
    # generate the testing set
    test_edges, test_edges_features_mtx, test_edges_labels = generate_test_set(graph_test, node_embeddings_testing, edge_operator)

    LR_clf = build_clf(train_edges_features_mtx, train_edges_labels)

    print("Edge Operator: {}".format(edge_operator))
    predict_edges_labels = pred_links(test_edges_features_mtx, LR_clf)
    precision, recall = precision_recall(list(predict_edges_labels), list(test_edges_labels))
    print('Precision: ', precision)
    print('Recall: ', recall)

[LibLinear]Edge Operator: Average
TP:  2359
TN:  3834
FP:  1495
FN:  2970
F1: 0.5137754546444517
[[3834 1495]
 [2970 2359]]
              precision    recall  f1-score   support

           0       0.56      0.72      0.63      5329
           1       0.61      0.44      0.51      5329

   micro avg       0.58      0.58      0.58     10658
   macro avg       0.59      0.58      0.57     10658
weighted avg       0.59      0.58      0.57     10658

Mean Average Precision: 0.549619714045685
Area Under ROC Curve: 0.5810658660161381
Precision:  0.6120913336792942
Recall:  0.4426721711390505
[LibLinear]Edge Operator: Hadamard
TP:  2963
TN:  2628
FP:  2701
FN:  2366
F1: 0.5390703174747566
[[2628 2701]
 [2366 2963]]
              precision    recall  f1-score   support

           0       0.53      0.49      0.51      5329
           1       0.52      0.56      0.54      5329

   micro avg       0.52      0.52      0.52     10658
   macro avg       0.52      0.52      0.52     10658
weighted a

### expdecay

In [54]:
exps = [np.exp(-i * 0.3) for i in range(1,17)]
node_embeddings_training = np.zeros((emb_list[0]).shape) 
for c,e in zip(emb_list[0:-2],exps[:-1]):
    node_embeddings_training += e * c 
node_embeddings_testing = np.zeros((emb_list[0]).shape) 
for c,e in zip(emb_list[0:-1],exps[:-1]):
    node_embeddings_testing += e * c 

In [55]:
for edge_operator in ['Average', 'Hadamard','Weighted-L1','Weighted-L2', 'Concat']:
    # generate the training set
    train_edges, train_edges_features_mtx, train_edges_labels = generate_train_set(graph_train, num_edge_sample, node_embeddings_training, edge_operator)
    # generate the testing set
    test_edges, test_edges_features_mtx, test_edges_labels = generate_test_set(graph_test, node_embeddings_testing, edge_operator)

    LR_clf = build_clf(train_edges_features_mtx, train_edges_labels)

    print("Edge Operator: {}".format(edge_operator))
    predict_edges_labels = pred_links(test_edges_features_mtx, LR_clf)
    precision, recall = precision_recall(list(predict_edges_labels), list(test_edges_labels))
    print('Precision: ', precision)
    print('Recall: ', recall)

[LibLinear]Edge Operator: Average
TP:  3208
TN:  3166
FP:  2163
FN:  2121
F1: 0.599626168224299
[[3166 2163]
 [2121 3208]]
              precision    recall  f1-score   support

           0       0.60      0.59      0.60      5329
           1       0.60      0.60      0.60      5329

   micro avg       0.60      0.60      0.60     10658
   macro avg       0.60      0.60      0.60     10658
weighted avg       0.60      0.60      0.60     10658

Mean Average Precision: 0.5585625234019678
Area Under ROC Curve: 0.5980484143366486
Precision:  0.5972816980078198
Recall:  0.6019891161568774
[LibLinear]Edge Operator: Hadamard
TP:  3127
TN:  2504
FP:  2825
FN:  2202
F1: 0.5543834766421417
[[2504 2825]
 [2202 3127]]
              precision    recall  f1-score   support

           0       0.53      0.47      0.50      5329
           1       0.53      0.59      0.55      5329

   micro avg       0.53      0.53      0.53     10658
   macro avg       0.53      0.53      0.53     10658
weighted a

### Email-EU - Equal Weekly
### load data

In [56]:
# load the training and testing graph
with open('../data/email_EU/email_equal_weekly_dir.pkl', 'rb') as file:
    graphs = pickle.load(file)
graph_train = graphs[-2]
graph_test = graphs[-1]

# parameters
num_edge_sample = 400

In [57]:
num_snapshots = len(graphs)
num_snapshots

74

In [59]:
emb_list = []
for i in range(num_snapshots):
    file = '../data/email_EU/embeddings/em-email-equal-week-' + str(i) + '.npy'
    node_embedding = np.load(file)
    emb_list.append(node_embedding)
print(len(emb_list))

74


In [60]:
node_embeddings_training = emb_list[-3]
node_embeddings_testing = emb_list[-2]

In [61]:
for edge_operator in ['Average','Hadamard','Weighted-L1','Weighted-L2', 'Concat']:
    # generate the training set
    train_edges, train_edges_features_mtx, train_edges_labels = generate_train_set(graph_train, num_edge_sample, node_embeddings_training, edge_operator)
    # generate the testing set
    test_edges, test_edges_features_mtx, test_edges_labels = generate_test_set(graph_test, node_embeddings_testing, edge_operator)

    LR_clf = build_clf(train_edges_features_mtx, train_edges_labels)

    print("Edge Operator: {}".format(edge_operator))
    predict_edges_labels = pred_links(test_edges_features_mtx, LR_clf)
    precision, recall = precision_recall(list(predict_edges_labels), list(test_edges_labels))
    print('Precision: ', precision)
    print('Recall: ', recall)

[LibLinear]Edge Operator: Average
TP:  242
TN:  1948
FP:  197
FN:  1903
F1: 0.1873065015479876
[[1948  197]
 [1903  242]]
              precision    recall  f1-score   support

           0       0.51      0.91      0.65      2145
           1       0.55      0.11      0.19      2145

   micro avg       0.51      0.51      0.51      4290
   macro avg       0.53      0.51      0.42      4290
weighted avg       0.53      0.51      0.42      4290

Mean Average Precision: 0.5057823725249693
Area Under ROC Curve: 0.5104895104895105
Precision:  0.55125284738041
Recall:  0.11282051282051282
[LibLinear]Edge Operator: Hadamard
TP:  1673
TN:  427
FP:  1718
FN:  472
F1: 0.604407514450867
[[ 427 1718]
 [ 472 1673]]
              precision    recall  f1-score   support

           0       0.47      0.20      0.28      2145
           1       0.49      0.78      0.60      2145

   micro avg       0.49      0.49      0.49      4290
   macro avg       0.48      0.49      0.44      4290
weighted avg   

### sum

In [62]:
for i in range(len(emb_list)):
    emb_list[i] = np.asfarray(emb_list[i],float)
node_embeddings_training = np.sum(np.asarray(emb_list[0:-2]),axis=0)
node_embeddings_testing = np.sum(np.asarray(emb_list[0:-1]),axis=0)

In [63]:
for edge_operator in ['Average', 'Hadamard','Weighted-L1','Weighted-L2', 'Concat']:
    # generate the training set
    train_edges, train_edges_features_mtx, train_edges_labels = generate_train_set(graph_train, num_edge_sample, node_embeddings_training, edge_operator)
    # generate the testing set
    test_edges, test_edges_features_mtx, test_edges_labels = generate_test_set(graph_test, node_embeddings_testing, edge_operator)

    LR_clf = build_clf(train_edges_features_mtx, train_edges_labels)

    print("Edge Operator: {}".format(edge_operator))
    predict_edges_labels = pred_links(test_edges_features_mtx, LR_clf)
    precision, recall = precision_recall(list(predict_edges_labels), list(test_edges_labels))
    print('Precision: ', precision)
    print('Recall: ', recall)

[LibLinear]Edge Operator: Average
TP:  1281
TN:  1369
FP:  776
FN:  864
F1: 0.609709662065683
[[1369  776]
 [ 864 1281]]
              precision    recall  f1-score   support

           0       0.61      0.64      0.63      2145
           1       0.62      0.60      0.61      2145

   micro avg       0.62      0.62      0.62      4290
   macro avg       0.62      0.62      0.62      4290
weighted avg       0.62      0.62      0.62      4290

Mean Average Precision: 0.5733075869196433
Area Under ROC Curve: 0.6177156177156178
Precision:  0.6227515799708313
Recall:  0.5972027972027972
[LibLinear]Edge Operator: Hadamard
TP:  1237
TN:  1017
FP:  1128
FN:  908
F1: 0.5485587583148559
[[1017 1128]
 [ 908 1237]]
              precision    recall  f1-score   support

           0       0.53      0.47      0.50      2145
           1       0.52      0.58      0.55      2145

   micro avg       0.53      0.53      0.53      4290
   macro avg       0.53      0.53      0.52      4290
weighted avg 

### expdecay

In [68]:
exps = [np.exp(-i * 0.3) for i in range(1,28)]
node_embeddings_training = np.zeros((emb_list[0]).shape) 
for c,e in zip(emb_list[0:-2],exps[:-1]):
    node_embeddings_training += e * c 
node_embeddings_testing = np.zeros((emb_list[0]).shape) 
for c,e in zip(emb_list[0:-1],exps[:-1]):
    node_embeddings_testing += e * c 

In [69]:
for edge_operator in ['Average', 'Hadamard','Weighted-L1','Weighted-L2', 'Concat']:
    # generate the training set
    train_edges, train_edges_features_mtx, train_edges_labels = generate_train_set(graph_train, num_edge_sample, node_embeddings_training, edge_operator)
    # generate the testing set
    test_edges, test_edges_features_mtx, test_edges_labels = generate_test_set(graph_test, node_embeddings_testing, edge_operator)

    LR_clf = build_clf(train_edges_features_mtx, train_edges_labels)

    print("Edge Operator: {}".format(edge_operator))
    predict_edges_labels = pred_links(test_edges_features_mtx, LR_clf)
    precision, recall = precision_recall(list(predict_edges_labels), list(test_edges_labels))
    print('Precision: ', precision)
    print('Recall: ', recall)

[LibLinear]Edge Operator: Average
TP:  1306
TN:  1276
FP:  869
FN:  839
F1: 0.6046296296296296
[[1276  869]
 [ 839 1306]]
              precision    recall  f1-score   support

           0       0.60      0.59      0.60      2145
           1       0.60      0.61      0.60      2145

   micro avg       0.60      0.60      0.60      4290
   macro avg       0.60      0.60      0.60      4290
weighted avg       0.60      0.60      0.60      4290

Mean Average Precision: 0.5611657155105432
Area Under ROC Curve: 0.6018648018648018
Precision:  0.6004597701149426
Recall:  0.6088578088578088
[LibLinear]Edge Operator: Hadamard
TP:  1529
TN:  696
FP:  1449
FN:  616
F1: 0.5969158696076519
[[ 696 1449]
 [ 616 1529]]
              precision    recall  f1-score   support

           0       0.53      0.32      0.40      2145
           1       0.51      0.71      0.60      2145

   micro avg       0.52      0.52      0.52      4290
   macro avg       0.52      0.52      0.50      4290
weighted avg 